In [1]:
import os,sys
import math
import pandas as pd
import numpy as np


os.getcwd()

'/home/fin/tmp/fis-ski'

In [2]:
import unicodecsv

def read_lines(reader):
    headers = reader.next()
    for x in reader:
        yield dict(zip(headers,x))

errorcount=0
errors = []

import re
numbersonly = re.compile('=([0-9]+)')

all_races = {}

ergebnisse =  pd.read_csv('fin_output_2.csv', dtype={"Total Time": object})

ergebnisse = ergebnisse.join(pd.DataFrame([x.split('.') for x in ergebnisse['subevent_Date']],columns=['day','month','year']))
ergebnisse['date'] = ergebnisse['subevent_Date']

#all_ergebnisse = ergebnisse.to_dict('records')

/home/fin/tmp/fis-ski/.venv/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,16,19,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


def sortgroup(xs, keyfun):
    import itertools
    return itertools.groupby(sorted(xs,key=keyfun),key=keyfun)

by_race = dict([(x,
                 list(sorted(
                    [z for z in y if z['Rank'] and z['Rank'].isdigit()]
                    , key=lambda e: int(e['Rank'],10))),
                )
                for (x,y,) in sortgroup(all_ergebnisse, lambda x: x['source_url'])
               ])

In [3]:
def extract_time(tt):
    if not type(tt)==float and tt.strip() and '?' not in tt:
        if ':' in tt:
            minutes,rest = tt.split(':',1)
        else:
            minutes = '0'
            rest = tt

        try:
            seconds,nths = re.split('[\.,]',rest)
        except(ValueError,e):
            seconds = rest
            nths = '0'

        minutes = int(minutes,10)
        seconds = int(seconds,10)
        nths_i = int(nths,10)
        nths_f = nths_i/(pow(10.0,len(nths)))
        finalseconds = minutes*60 + seconds + nths_f


        return finalseconds
    return np.NaN

ergebnisse['mytime'] = [extract_time(x) for x in ergebnisse['Total Time']]


ergebnisse['diff'] = [x[1]['mytime'] - ergebnisse[ergebnisse['source_url']==x[1]['source_url']].iloc[0]['mytime'] for x in ergebnisse.iterrows()]

ergebnisse['diff']

pointreplaced = 0

for raceurl,finishers in by_race.iteritems():

    last = None
    for f in finishers:
        f['rang'] = int(f['Rank'],10)



            f['diff'] = f['mytime']-finishers[0]['mytime']
            if last and hasattr(last,'mytime'):
                if f['diff']<last['diff']:
                    print('time travel?')
                    print(f['Total Time'],f['mytime'],last['Total Time'],last['mytime'],f['source_url'])
                    print(seconds)
                    print(nths, len(nths), nths_i, nths_f)
    
        last = f

In [4]:
df = ergebnisse
#df.to_csv('tmp.csv',encoding='utf-8')
df = df[~df['subevent_Discipline'].str.contains('training')]
df = df[df['subevent_Category']=='WSC']
#df_with_diffs = df[pd.isnull(df['diff'])]
df_with_rank = df[~pd.isnull(df['Rank'])]

In [5]:
df_with_rank[df_with_rank['month']==8].to_csv('tmp.csv',encoding='utf-8')

In [6]:
disziplinnamen = {
    'combined': 'Kombination',
    'alpine combined': 'Kombination',
    'downhill': 'Abfahrt',
    'slalom': 'Slalom',
    'giant slalom': 'Riesentorlauf',
    'super g': 'Super-G',
    'team': 'Team',
    'super combined': 'Kombination'
}
df_with_rank['subevent_Discipline'] = [disziplinnamen[x.lower()] for x in df_with_rank['subevent_Discipline']]

/home/fin/tmp/fis-ski/.venv/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
stockerl = df_with_rank[(df_with_rank['Rank'].convert_objects(convert_numeric=True)<=3)
             & ~df_with_rank['subevent_Discipline'].str.contains('combin')
            ][['Name','Rank','Nation','subevent_Discipline','subevent_Gender','year']]
for k,v in stockerl.groupby('subevent_Discipline'):
    fn = k.lower().replace(' ','_')
    print(fn)
    v.rename(columns=lambda x: x.replace('subevent_','').lower()).to_csv('out/'+fn+'.csv',encoding='utf-8',index=False)

abfahrt
kombination
riesentorlauf
slalom
super-g
team


/home/fin/tmp/fis-ski/.venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


with open('ergebnisse_w_diffs.csv','w') as f:
    cw = unicodecsv.writer(f)
    f = by_race.values()[0][0]
    fk = [x for x in f.keys() if x!='race' and x!='source_url']
    rk = []#f['race'].keys()
    headers = fk+rk
    cw.writerow(headers)
    for y in by_race.values():
        for x in y:
            if x['Total Time'].strip():
                cw.writerow(
                    [x[k] for k in fk] + [x['race'][k] for k in rk])

import itertools
all_diffs = [x for x in itertools.chain(*by_race.values()) if x.get('diff',None)]

import collections

MAX_SECONDS = 4

all_fitting = {k:[{'Rank': x['Rank'],
                   'Name': x['Name'],
                   'Time': x['Total Time'],
                   'diff': x['diff'],
                   'time': x['mytime'],
                   'race': x,
                  } for x in vv if x.get('diff',None) is not None and x['diff']<=MAX_SECONDS] for k,vv in by_race.iteritems()
               if vv}

result = []
for k,v in all_fitting.iteritems():
    if not v:
        continue
    r = {}
    r.update(v[0]['race'])
    r['year'] = v[0]['race']['year']
    r['month'] = v[0]['race']['month']
    r['ranks'] = v
    result.append(r)
#result = all_fitting.values()

last_season_races = [x for x in result
 if ('2016'==x['year'] and int(x['month'],10)>6)
 or (x['year']=='2017' and int(x['month'],10)<=6)]

df_with_rank[df_with_rank['year']==2016].to_csv('tmp.csv',encoding='utf-8')

for race in last_season_races:
    tmp = [x['diff'] for x in race['ranks'] if int(x['Rank'],10)<=5]
    race['knappheit'] = sum(tmp)/len(tmp)
    race['diffzweiter'] = race['ranks'][1]['diff']
    race['finishersinnsecs'] = len([x for x in race['ranks'] if x['diff']<=MAX_SECONDS])

by_disc = dict((x,list(y),) for x,y in sortgroup(last_season_races,
                                                 lambda x: '%s (%s)' % (x['disziplin'],x['gender'],)))

def mean(numbers):
    return sum(numbers)/(1.0*len(numbers))

MAX_RANK = 15

with open('out.csv','w') as f, open('debug.csv','w') as f2:
    cw = unicodecsv.writer(f)
    cw2 = unicodecsv.writer(f2)
    
    cw.writerow(['Disziplin','Rennen']+
               list(itertools.chain.from_iterable([['Platz %s'%i, 'Name %s'%i] for i in xrange(1,MAX_RANK+1)])))

    for disc,discraces in by_disc.iteritems():
        closest = sorted(discraces, key=lambda x:x['diffzweiter'])[0]
        cw.writerow([disc,'closest (%s / %s)'%(closest['ort'],closest['date'],)]+
                    list(itertools.chain.from_iterable([
                     [round(x['diff'],2),x['Name']]
                      for x in closest['ranks'][:MAX_RANK+1]
                     ]
                   )
                    ))
        all_fitting = [[[x['diff'],x['Name']] for x in reduce(lambda x,y: x+y, [z['ranks'] for z in discraces],[]) if int(x['Rank'],10)==y] for y in xrange(1,MAX_RANK+1)]
        average = [mean([y[0] for y in x]) for x in all_fitting]
        cw.writerow([disc,'mean']+
                    list(itertools.chain.from_iterable([
                        [round(x,2),'%d. Platz' % (i+1)] for i,x in enumerate(average)
                    ]))
           )
        for x in all_fitting:
            diffs = [y[0] for y in x]
            x_sorted = sorted(x, key=lambda y: y[0])
            cw2.writerow([disc]+['%f:%s'%(y[0],y[1],) for y in x_sorted]+['||',mean(diffs),sorted(diffs)[int(math.floor(len(x)/2))]])

x

races_with_time = set([x['source_url'] for x in all_ergebnisse if 'rang' in x and x['rang'] and 'time' in x])
races_witout_time = set([x['source_url'] for x in all_ergebnisse if  'rang' in x and  x['rang'] and not 'time' in x])

len(races_with_time & races_witout_time)

auswertungen = {
 '1': lambda x: x['Rank']=='1',
 '1 at': lambda x: x['Rank']=='1' and x['Nation']=='AUT',
 '2': lambda x: x['Rank']=='2',
 '2 at': lambda x: x['Rank']=='2' and x['Nation']=='AUT',
 '3': lambda x: x['Rank']=='3',
 '3 at': lambda x: x['Rank']=='3' and x['Nation']=='AUT',
}

all_by_disziplin = {k: list(v) for k,v in sortgroup(all_ergebnisse,
                                                    lambda x: 
                                                    '%s (%s)' % (x['subevent_Discipline'],x['subevent_Gender'],)
                                                   )}

auswertungen_by_disziplin = {
    k: {aname: len(list(x for x in v if aprop(x)))
        for aname, aprop
        in auswertungen.iteritems()
       }
    for k,v in all_by_disziplin.iteritems()
}

with open('allraces.csv','w') as f:
    cw = unicodecsv.writer(f)
    cw.writerow(['Kategorie','Platz', 'Wert'])
    for k,v in auswertungen_by_disziplin.iteritems():
        for k2, v2 in v.iteritems():
            cw.writerow([k,k2,v2])

all_ergebnisse[0]

In [8]:
import datetime
print('success',datetime.datetime.now())

('success', datetime.datetime(2017, 2, 15, 15, 51, 36, 986776))
